# Comparing different Neural Networks

In [19]:
using LmaPredict, Flux, Statistics, StatsBase, ScikitLearn

[ Info: Precompiling ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324]
┌ Warning: Module StatsBase with build ID fafbfcfd-d044-e7f2-0002-6fc13fc7eaf9 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324].
[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
┌ Warning: Module Missings with build ID fafbfcfd-b2ef-bec6-0002-6fbf642e491f is missing from the cache.
│ This may mean Missings [e1d29d7a-bbdc-5cf2-9ac0-f12de2c33e28] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0].
[ Info: Precompiling InlineStrings [842dd82b-1e85-43dc-bf29-5d0ee9dffc48]
Precom

## Reading the data

In [2]:
const path_config = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Daten Simon/dat"
const path_plot = "../plots"

"../plots"

In [3]:
fname = readdir(path_config)[2:5001]
idx = sortperm( parse.(Int64, fname))
fname = fname[idx]
em_n = "VV"

cnfgarr = Vector{LMAConfig}(undef, 0)
for f in fname
    push!(cnfgarr, get_LMAConfig(joinpath(path_config, f), "g5-g5", em=em_n, bc=false))
end

## Splitting data in training and test sets

In [4]:
NCNFG = length(cnfgarr)
train_size = 500
test_size = NCNFG - train_size

TSRC = "24"
TVALS = length(cnfgarr[1].data["rr"][TSRC]) - 1
if em_n == "PA"
    EIGVALS = 32
else 
    EIGVALS = 64
end

eigvals_data_train = Array{Float64}(undef, EIGVALS, train_size)
rr_data_train = Array{Float64}(undef, TVALS, train_size)
ee_data_train = Array{Float64}(undef, TVALS, train_size)
re_data_train = Array{Float64}(undef, TVALS, train_size)

eigvals_data_test = Array{Float64}(undef, EIGVALS, test_size)
rr_data_test = Array{Float64}(undef, TVALS, test_size)
ee_data_test = Array{Float64}(undef, TVALS, test_size)
re_data_test = Array{Float64}(undef, TVALS, test_size)

for (k, dd) in enumerate(getfield.(cnfgarr, :data)[1:train_size])
    eigvals_data_train[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_train[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    #ee_data_train[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_train[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]

    ee_all_TSRC = Matrix{Float64}(undef, TVALS, TVALS)
    for ee_TSRC in 0:TVALS-1
        ee_all_TSRC[:,ee_TSRC+1] = getindex(getindex(dd, "ee"), "$ee_TSRC")[2:end]
    end
    ee_data_train[:,k] = mean(ee_all_TSRC, dims=2)
end
for (k, dd) in enumerate(getfield.(cnfgarr, :data)[train_size+1:NCNFG])
    eigvals_data_test[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_test[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    #ee_data_test[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_test[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]

    ee_all_TSRC = Matrix{Float64}(undef, TVALS, TVALS)
    for ee_TSRC in 0:TVALS-1
        ee_all_TSRC[:,ee_TSRC+1] = getindex(getindex(dd, "ee"), "$ee_TSRC")[2:end]
    end
    ee_data_test[:,k] = mean(ee_all_TSRC, dims=2)
end

In [47]:
k = 5

k_fold_DataSet = Array{Matrix{Float64}}(undef, k)
for i in 1:k
    

1×1 Matrix{Int64}:
 3

In [48]:
k_fold_DataSet

5-element Vector{Matrix{Float64}}:
    [3.0;;]
 #undef
 #undef
 #undef
 #undef

In [44]:
input_data_train_standardized[CrossValidation.KFold(500, n_folds=5)[1]]

LoadError: ArgumentError: invalid index: ([101, 102, 103, 104, 105, 106, 107, 108, 109, 110  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]) of type Tuple{Vector{Int64}, Vector{Int64}}

In [43]:
CrossValidation.KFold(500, n_folds=5)

5-element Vector{Tuple{Vector{Int64}, Vector{Int64}}}:
 ([101, 102, 103, 104, 105, 106, 107, 108, 109, 110  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100])
 ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200])
 ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], [201, 202, 203, 204, 205, 206, 207, 208, 209, 210  …  291, 292, 293, 294, 295, 296, 297, 298, 299, 300])
 ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], [301, 302, 303, 304, 305, 306, 307, 308, 309, 310  …  391, 392, 393, 394, 395, 396, 397, 398, 399, 400])
 ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  391, 392, 393, 394, 395, 396, 397, 398, 399, 400], [401, 402, 403, 404, 405, 406, 407, 408, 409, 410  …  491, 492, 493, 494, 495, 496, 497, 498, 

## Defining training and test data

In [5]:
input_length = 2*TVALS + EIGVALS
output_length = TVALS

#input_shape_train = vcat(1 ./ eigvals_data_train, ee_data_train, rr_data_train)
input_shape_train = vcat(ee_data_train, rr_data_train)
output_shape_train = re_data_train

#input_shape_test = vcat(1 ./ eigvals_data_test, ee_data_test, rr_data_test)
input_shape_test = vcat(ee_data_test, rr_data_test)
output_shape_test = re_data_test;

### Input data, normalized and standardized

In [6]:
max_input_train = maximum(input_shape_train)
min_input_train = minimum(input_shape_train)

mean_input_train = mean(input_shape_train, dims=ndims(input_shape_train))
std_input_train = std(input_shape_train, dims=ndims(input_shape_train))

input_data_train_normalized = (input_shape_train .- max_input_train) ./ (max_input_train - min_input_train)
input_data_train_standardized = (input_shape_train .- mean_input_train) ./ std_input_train

input_data_test_normalized = (input_shape_test .- max_input_train) ./ (max_input_train - min_input_train)
input_data_test_standardized = (input_shape_test .- mean_input_train) ./ std_input_train;

### Output data, normalized and standardized

In [7]:
max_output_train = maximum(output_shape_train)
min_output_train = minimum(output_shape_train)

mean_output_train = mean(output_shape_train, dims=ndims(output_shape_train))
std_output_train = std(output_shape_train, dims=ndims(output_shape_train))

output_data_train_normalized = (output_shape_train .- max_output_train) ./ (max_output_train - min_output_train)
output_data_train_standardized = (output_shape_train .- mean_output_train) ./ std_output_train;

## Describing different Neural Networks

In [8]:
activation_function = NNlib.tanh

models = [
    Chain(
    Dense(94 => 400, activation_function),               
    Dropout(0.8),
    Dense(400 => 47, identity),                     
    ) 
] |> f64;

In [9]:
function loss(flux_model, x, y)
    ŷ = flux_model(x)
    Flux.mse(ŷ, y, agg=sum)
end

loss (generic function with 1 method)

In [10]:
using Flux:params

optimizer = Flux.AdaMax(0.001)
loss_function = Flux.mse
loss_discription = "MSE"

epochs = 95
batch_size = 64

percentages_bc = [0.0, 0.01, 0.02, 0.05, 0.1, 0.12]
n_configs_bc = Int.(4500 .* percentages_bc)

loader = Flux.DataLoader((input_data_train_standardized, output_data_train_standardized), batchsize=batch_size, shuffle=true)

for model in models
    parameters = 0
    layers = params(model)
    for layer in layers
        parameters += length(hcat(layer...))
    end

    outputDirectory = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Tool Allesandro/repo/LmaPredict/benchmarks/ee+rr/$parameters"

    optim = Flux.setup(optimizer, model)
    
    function training()
        losses = []
        for epoch in 1:epochs
            for (x, y) in loader
                grads = gradient(m -> loss(m, x, y), model)
                Flux.update!(optim, model, grads[1])
                push!(losses,loss(model, x, y))
            end
        end
        return losses
    end

    losses = training()
    
    out_of_sample_predictions = (model(input_data_test_standardized) .* std_output_train) .+ mean_output_train

    analyse_predictions(
        out_of_sample_predictions,
        output_shape_test,
        TSRC,
        EIGVALS,
        model,
        optimizer,
        loss_function,
        loss_discription,
        epochs,
        batch_size,
        losses,
        outputDirectory
    )
end